# WINNING HORSE - Limpieza de datos

## Importación de librerías

In [2]:
import pandas as pd
import numpy as np
from datetime import date, datetime, time

## Datasets

In [3]:
df2022 = pd.read_csv("./../data/Results_Global_2020_winnersTop_nuevo.csv")
df2021 = pd.read_csv("./../data/Results_Global_2021_winnersTop_nuevo.csv")
df2020 = pd.read_csv("./../data/Results_Global_2022_winnersTop_nuevo.csv")
dfResults = pd.concat([df2020, df2021, df2022], ignore_index=True)

## Limpieza de datos - Primera parte
Se limpian las columnas que no se van a utilizar en el análisis

In [4]:
dropColumns = [
    "STATUS",
    "AGELIMIT",
    "SEXTYPE",
    "LATITUDE",
    "LONGITUDE",
    "YEAR",
    "COUNTY",
    "GROUNDTEXT",
    "LEADINGEARNERHORSE",
    "ENTRYTYPE",
    "PRIZEAMOUNT",
    "POSITIONFINISHTIME",
    "RESULTFINISHPOS",
    "TOTALOWNERPRIZEMONEYWON",
    "ownername",
    "TRAINERSTYLE",
    "LICENCETYPE",
    "LOCATION",
    "AGEYEAR"
]
dfResults.drop(dropColumns, axis=1, inplace=True)
print(dfResults.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208326 entries, 0 to 208325
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   REGION                208326 non-null  object 
 1   TRACKHANDEDNESS       208326 non-null  object 
 2   RACEDATE              208326 non-null  object 
 3   RACETIME              208326 non-null  object 
 4   DISTANCE              208326 non-null  int64  
 5   TIER                  204038 non-null  object 
 6   MAJOREVENT            14996 non-null   float64
 7   GROUND                192512 non-null  float64
 8   GOINGSTICK            115968 non-null  float64
 9   WEATHER               43429 non-null   object 
 10  DIVISIONSEQUENCE      208326 non-null  int64  
 11  NAME                  208326 non-null  object 
 12  DATEOFBIRTH           208326 non-null  object 
 13  CLOTHNUMBER           208326 non-null  int64  
 14  WEIGHTVALUE           208326 non-null  int64  
 15  

## Características del dataset

Reemplazar los ratios de divisiones por 0 y evaluamos para convertirlos en numéricos y eliminamos la columna de literales

In [5]:
dfResults["EVAL_BETTINGRATIO"] = dfResults["BETTINGRATIO"].replace({"0/0":"0/1"}).apply(eval)

Cambiar las fechas a formato datetime para poder operar con ellas, y combinación de date y time en una sola variable

In [6]:
dfResults["DATEOFBIRTH"] = pd.to_datetime(dfResults["DATEOFBIRTH"])
dfResults["RACEDATE"] = pd.to_datetime(dfResults["RACEDATE"] + ' ' + dfResults["RACETIME"])

Creación del dato EdadYears que refleja la edad exacta en años del caballo el día de la carrera

In [7]:
edades = dfResults["RACEDATE"] - dfResults["DATEOFBIRTH"]
dfResults["EdadYears"] = edades / np.timedelta64(1, 'Y')

Obtenemos las estaciones del año por carrera, nos permite generalizar más que el día y el mes, por si un caballo tiene mejor rendimiento en estaciones cálidas o frías

In [8]:
anio = 2000;
datesSeasons = [
    ("Winter", (date(anio, 1, 1), date(anio, 3, 19))),
    ("Spring", (date(anio, 3, 20), date(anio, 6, 20))),
    ("Summer", (date(anio, 6, 21), date(anio, 9, 22))),
    ("Fall", (date(anio, 9, 23), date(anio, 12, 21))),
    ("Winter", (date(anio, 12, 22), date(anio, 12, 31))),
]
def get_season(now):
    now = now.date()
    now = now.replace(year=anio)
    return next(a for a, (start, end) in datesSeasons if start <= now <= end)

dfResults["SEASONS"] = dfResults["RACEDATE"].apply(lambda x: get_season(x))

Obtenemos el rango de horas según la hora de salida (mañana, mediodia, tarde y noche)

In [9]:
def horario(hora):
    if hora.time() >= time(7) and hora.time() < time(11):
        return "Morning"
    elif hora.time() >= time(11) and hora.time() < time(15):
        return "Midday"
    if hora.time() >= time(15) and hora.time() < time(19):
        return "Afternoon"
    else:
        return "Night"

dfResults["HORARIO"] = dfResults["RACEDATE"].apply(lambda x: horario(x))

Sustituir valores nulos de Major event y de Ground
- ***MAJOR EVENT*** : Los eventos con valor nulo se catalogan con valor 0 (no importantes)
- ***GROUND*** : La condición de terreno con valor nulo se cataloga con la media de los valores conocidos
- ***GOING STICK*** : La condición de agarre del terreno con valor nulo se cataloga con la media de los valores conocidos

In [10]:
# Rellenamos los valores de MajorEvent = NaN con valor 0
dfResults["MAJOREVENT"].fillna(0, inplace=True)

#Rellenar los valores de Ground = NaN con la media de la columna
dfResults["GROUND"].fillna(dfResults["GROUND"].mean(), inplace=True)

#Rellenar los valores de GoingStick = NaN con la media de la columna
dfResults["GOINGSTICK"].fillna(dfResults["GOINGSTICK"].mean(), inplace=True)

### Manga de caballos
Creamos un identificador único para las mangas, con el fin de sacar el mínimo betting ratio por manga y así calcular los favoritos
*(Funciones de larga duración)*

In [11]:
#Por cada fecha, buscamos las horas y por cada hora, buscamos la manga y por cada manga añadimos un identificador para identificar la manga
id_manga = 0
fechas = dfResults["RACEDATE"].unique()
for fecha in fechas:
    mangas = dfResults[dfResults["RACEDATE"] == fecha]["DIVISIONSEQUENCE"].unique()
    for manga in mangas:
        dfResults.loc[(dfResults["RACEDATE"] == fecha) & (dfResults["DIVISIONSEQUENCE"] == manga), "IDMANGA"] = id_manga
        id_manga += 1

#Creamos una columna con el valor mínimo del bettingRatio por cada manga y el mejor tiempo
for manga in dfResults["IDMANGA"]:
    dfResults.loc[dfResults["IDMANGA"] == manga, "MIN_BETTINGRATIO"] = dfResults[dfResults["IDMANGA"] == manga]["EVAL_BETTINGRATIO"].min()

#Asignamos la columna de favorito a True de la carrera si el BettingRatio es igual al mínimo
dfResults["FAVORITE"] = dfResults["EVAL_BETTINGRATIO"] ==  dfResults["MIN_BETTINGRATIO"]

### WEATER
Comprobamos la variable WEATHER sobre el número de nulos, por si aporta suficiente valor. Dado que el número de nulos es muy elevado, se decide eliminar la columna en vez de completar los nulos.

In [12]:
# obtener el porcentaje de nulos de la columna Weather
print("Porcentaje de nulos de la columna WEATHER: ", dfResults["WEATHER"].isnull().sum() / len(dfResults["WEATHER"]) * 100, "%")

Porcentaje de nulos de la columna WEATHER:  79.15334619778616 %


## Limpieza de datos - Segunda parte

In [13]:
dropColumns = [
    # Eliminamos las columnas que no necesitamos tras el cálculo de la nueva manga
    "EVAL_BETTINGRATIO",
    "MIN_BETTINGRATIO",
    "RACEDATE",
    "IDMANGA",
    "DIVISIONSEQUENCE",

    # Dado que el número de nulos es muy elevado, se decide eliminar la columna en vez de completar los nulos
    "WEATHER",

    # Eliminamos las columnas que no nos aportan información
    "BETTINGRATIO",
    "RACETIME",
    "DATEOFBIRTH",
]
dfResults.drop(dropColumns, axis=1, inplace=True)
print(dfResults.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208326 entries, 0 to 208325
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   REGION                208326 non-null  object 
 1   TRACKHANDEDNESS       208326 non-null  object 
 2   DISTANCE              208326 non-null  int64  
 3   TIER                  204038 non-null  object 
 4   MAJOREVENT            208326 non-null  float64
 5   GROUND                208326 non-null  float64
 6   GOINGSTICK            208326 non-null  float64
 7   NAME                  208326 non-null  object 
 8   CLOTHNUMBER           208326 non-null  int64  
 9   WEIGHTVALUE           208326 non-null  int64  
 10  RACECRITERIARACETYPE  208326 non-null  object 
 11  ENTRYNAME             208326 non-null  object 
 12  CHAMPIONSHIPTYPE      208326 non-null  object 
 13  FULLNAME              208326 non-null  object 
 14  TOP1                  208326 non-null  int64  
 15  

### Dummies
Generamos Campos Dummies para las variables categóricas

In [22]:
dummiesColumns = ["TRACKHANDEDNESS", "RACECRITERIARACETYPE", "SEASONS", "HORARIO"]
dummiesColumnsNames = ["TrackHandedness", "RaceCriteria", "Seasons", "Schedule"]
dummies = pd.get_dummies(dfResults[dummiesColumns], dummiesColumnsNames, drop_first=True)
dfResults = pd.concat([dfResults, dummies], axis=1)
dfResults.drop(dummiesColumns, axis=1, inplace=True)
dfResults

,Region,Distance,Category,MajorEvent,GroundCondition,Stick,HorseName,StartingStall,Weight,JockeyName,...,EdadYears,FAVORITE,TrackHandedness_Left-Handed,TrackHandedness_Right-Handed,RaceCriteria_JUMP,Seasons_Spring,Seasons_Summer,Seasons_Winter,Schedule_Midday,Schedule_Night
0,SOUTH,2646,-100,1.0,8.192897,6.898805,Sonny Liston (IRE),12,128,Tom Marquand,...,3.344867,False,1,0,0,1,0,0,0,0
1,SOUTH,2646,-100,1.0,8.192897,6.898805,Grand Alliance (IRE),5,128,Daniel Tudhope,...,3.180592,False,1,0,0,1,0,0,0,0
2,SOUTH,2646,-100,1.0,8.192897,6.898805,Desert Crown (GB),2,128,Richard Kingscote,...,3.259992,True,1,0,0,1,0,0,0,0
3,SOUTH,2646,-100,1.0,8.192897,6.898805,Hoo Ya Mal (GB),6,128,David Probert,...,3.136786,False,1,0,0,1,0,0,0,0
4,SOUTH,2646,-100,1.0,8.192897,6.898805,Grand Alliance (IRE),5,128,Daniel Tudhope,...,3.180592,False,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208321,NORTH,1100,-100,0.0,6.000000,5.300000,Iris Dancer (GB),7,122,Lewis Edmunds,...,2.676475,False,1,0,0,0,0,0,1,0
208322,NORTH,1100,-100,0.0,6.000000,5.300000,Go Go Luna (GB),11,112,Paula Muir,...,2.676475,False,1,0,0,0,0,0,1,0
208323,NORTH,1312,-100,0.0,7.000000,4.800000,Sharrabang (GB),3,130,Faye McManoman,...,4.453643,False,1,0,0,0,0,0,0,0
208324,NORTH,1312,-100,0.0,7.000000,4.800000,Lili Wen Fach (IRE),10,125,Hollie Doyle,...,3.695243,False,1,0,0,0,0,0,0,0


Comprobamos que los valores objetivo solo tienen dos valores 1 o 0

In [23]:
dfResults["TOP1"].value_counts()

0    183818
1     24508
Name: TOP1, dtype: int64

In [18]:
dfResults["TOP3"].value_counts()

0    126763
1     81563
Name: TOP3, dtype: int64

In [19]:
dfResults["TOP5"].value_counts()

1    125871
0     82455
Name: TOP5, dtype: int64

Renombramos las columnas para un mejor entendimiento

In [24]:
dfResults.rename(columns={'REGION': 'Region',
                   'DISTANCE': 'Distance',
                   'TIER': 'Category',
                   'MAJOREVENT': 'MajorEvent',
                   'GROUND': 'GroundCondition',
                   'GOINGSTICK': 'Stick',
                   'NAME': 'HorseName',
                   'CLOTHNUMBER': 'StartingStall',
                   'WEIGHTVALUE': 'Weight',
                   'ENTRYNAME': 'JockeyName',
                   'CHAMPIONSHIPTYPE': 'ChampionshipType',
                   'FULLNAME': 'OwnerName'}, inplace=True)
dfResults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208326 entries, 0 to 208325
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Region                        208326 non-null  object 
 1   Distance                      208326 non-null  int64  
 2   Category                      204038 non-null  object 
 3   MajorEvent                    208326 non-null  float64
 4   GroundCondition               208326 non-null  float64
 5   Stick                         208326 non-null  float64
 6   HorseName                     208326 non-null  object 
 7   StartingStall                 208326 non-null  int64  
 8   Weight                        208326 non-null  int64  
 9   JockeyName                    208326 non-null  object 
 10  ChampionshipType              208326 non-null  object 
 11  OwnerName                     208326 non-null  object 
 12  TOP1                          208326 non-nul

## Guardar el dataset

In [25]:
pd.DataFrame.to_csv(dfResults, "../data/winning_horse_limpieza.csv", index=False)